In [ ]:
'''
Author: Vighnesh Ghantasala 
Github: https://github.com/VighneshGhantasala/
'''

def build_form(align='center',input_tags=1,fname='sample.html'):
    f = open(fname,'w+')    
    form = "<div align=\'" + align + "\'><h1>My Form</h1><form>"
    if input_tags > 0 :
        form += "<br><br><input type='text' />" * input_tags
        form += "<br><input type='submit' value='submit'>"
    else:
        form += "<br> <h1>NO inputs detected</h1>"
    form += "</form></div>" 
    f.write("<html><body>"+ form+"</body></html>")
    f.close()    


In [ ]:
import cv2,os,webbrowser
import numpy as np

align = 'center'

def extract_form(img):
    global align
    inp = 0
    canny = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,5)
    canny = cv2.dilate(canny,(3,3))
    cont, _ = cv2.findContours(canny,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for c in cont:
        approx = cv2.approxPolyDP(c,0.01 * (cv2.arcLength(c,True)),True)
        cv2.drawContours(img,[approx],-1,(255,255,255),5)
        x,y,w,h = cv2.boundingRect(approx)
    if x+w <=200 :
        align = 'left'
    elif x+w <=400 :
        align = 'center'
    else:
        align = 'right'
    
    pts1 = np.float32([[x,y],[x+w,y],[x,y+h],[x+w,y+h]])
    pts2 = np.float32([[0,0],[x+w,0],[0,y+h],[x+w,y+h]])

    per = cv2.getPerspectiveTransform(pts1,pts2)
    res = cv2.warpPerspective(img,per,dsize=(x+w,y+h))
    cny = cv2.adaptiveThreshold(res,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,5)
    cny = cv2.dilate(cny,(5,5))
    cc, _ = cv2.findContours(cny,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for c in cc:
        approx = cv2.approxPolyDP(c,0.01 * (cv2.arcLength(c,True)),True)
        if len(approx) >= 4:
            print(approx[0][0],cv2.contourArea(c))
            cv2.drawContours(res,[approx],-1,(0,0,255))
            inp +=1
        else:
            inp =0
            return align ,inp
    return align,inp


path = os.listdir(r"path\to\source\images\folder")

for i in path:
    if i.endswith('png') or i.endswith('jpg'): # add more image formats if you need
        fname = os.path.splitext(i)[0] + ".html"
        img = cv2.imread(i)
        img = cv2.resize(img,(600,300))
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        align,inp = extract_form(gray)
        build_form(align,inp,fname)
        webbrowser.open(fname)